<a href="https://colab.research.google.com/github/Colsai/Coursera-Data-Science-Practice/blob/master/DATA602_Small_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chicago Public Schools Revisited
## Machine Learning Models that can predict school quality based on graduation rates.
## Grouping by clusters, and creating analyzing performance with KNN, etc.
## DATA602 | Krishnamoorthy

In [ ]:
#Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import time
from statsmodels.graphics.gofplots import qqplot
import sklearn

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Import each dataset (Chicago Public Schools)
start_time = time.time()

cps_1516_df = pd.read_csv("https://data.cityofchicago.org/api/views/fvrx-esxp/rows.csv?accessType=DOWNLOAD&bom=true&format=true")

dur = time.time()- start_time

print(f"{round(dur,3)}s")

4.685s


In [ ]:
#Small function for displaying many elements in a list in-line
def show_all(input_list, line_len = 3):
    num_in_line = 0

    for elem in input_list:
        if num_in_line > line_len - 1:
            print(f"")
            print(elem, end = ', ')
            num_in_line = 0
        
        else:
            print(elem, end = ', ')
        
        num_in_line += 1

In [ ]:
#Introduce the four years' datasets
all_dfs = [cps_1516_df]

for dataframe in all_dfs:
    print(f"elem:{dataframe.size} | cols:{(dataframe.shape)}")

elem:102510 | cols:(670, 153)
elem:106421 | cols:(661, 161)
elem:107743 | cols:(661, 163)
elem:119028 | cols:(654, 182)


In [ ]:
#Drop "," and replace Columns as 'int' for cps_1718 and cps_1819
cps_1718_df['School_ID'] = cps_1718_df['School_ID'].apply(lambda x:int(x.replace(",","")))
cps_1819_df['School_ID'] = cps_1819_df['School_ID'].apply(lambda x:int(x.replace(",","")))

#Merge the datasets together. (Outer join to keep all data)
df_1517 = cps_1516_df.merge(cps_1617_df, how = "outer", on="School_ID", suffixes=('_1516','_1617'))
df_1719 = cps_1718_df.merge(cps_1819_df, how = "outer",  on = "School_ID",suffixes = ('_1718','_1819'))

#Join the data again
full_df = df_1517.merge(df_1719, how = "outer", on="School_ID")

In [ ]:
full_df

,School_ID,Short_Name_1516,Long_Name_1516,School_Type_1516,Primary_Category_1516,Address_1516,City_1516,State_1516,Zip_1516,Phone_1516,Fax_1516,CPS_School_Profile_1516,Website_1516,Progress_Report_Year_1516,Blue_Ribbon_Award_Year_1516,Excelerate_Award_Gold_Year_1516,Spot_Light_Award_Year_1516,Improvement_Award_Year_1516,Excellence_Award_Year_1516,Student_Growth_Rating_1516,Student_Growth_Description_1516,Growth_Reading_Grades_Tested_Pct_ES_1516,Growth_Reading_Grades_Tested_Label_ES_1516,Growth_Math_Grades_Tested_Pct_ES_1516,Growth_Math_Grades_Tested_Label_ES_1516,Student_Attainment_Rating_1516,Student_Attainment_Description_1516,Attainment_Reading_Pct_ES_1516,Attainment_Reading_Lbl_ES_1516,Attainment_Math_Pct_ES_1516,Attainment_Math_Lbl_ES_1516,Culture_Climate_Rating_1516,Culture_Climate_Description_1516,School_Survey_Student_Response_Rate_Pct_1516,School_Survey_Student_Response_Rate_Avg_Pct_1516,School_Survey_Teacher_Response_Rate_Pct_1516,School_Survey_Teacher_Response_Rate_Avg_Pct_1516,Healthy_School_Certification_1516,Healthy_School_Certification_Description_1516,Creative_School_Certification_1516,...,College_Persistence_School_Pct_Year_2_1819,College_Persistence_CPS_Pct_Year_2_1819,College_Persistence_School_Pct_Year_1_1819,College_Persistence_CPS_Pct_Year_1_1819,Progress_Toward_Graduation_Year_1_1819,Progress_Toward_Graduation_Year_2_1819,State_School_Report_Card_URL_1819,Mobility_Rate_Pct_1819,Chronic_Truancy_Pct_1819,Empty_Progress_Report_Message_1819,School_Survey_Rating_Description_1819,Supportive_School_Award_1819,Supportive_School_Award_Desc_1819,Parent_Survey_Results_Year_1819,School_Latitude,School_Longitude,PSAT_Grade_9_Score_School_Avg,PSAT_Grade_10_Score_School_Avg,SAT_Grade_11_Score_School_Avg,SAT_Grade_11_Score_CPS_Avg,Growth_PSAT_Grade_9_School_Pct,Growth_PSAT_Grade_9_School_Lbl,Growth_PSAT_Reading_Grade_10_School_Pct,Growth_PSAT_Reading_Grade_10_School_Lbl,Growth_SAT_Grade_11_School_Pct,Growth_SAT_Grade_11_School_Lbl,Attainment_PSAT_Grade_9_School_Pct,Attainment_PSAT_Grade_9_School_Lbl,Attainment_PSAT_Grade_10_School_Pct,Attainment_PSAT_Grade_10_School_Lbl,Attainment_SAT_Grade_11_School_Pct,Attainment_SAT_Grade_11_School_Lbl,Attainment_All_Grades_School_Pct,Attainment_All_Grades_School_Lbl,Growth_PSAT_Math_Grade_10_School_Pct,Growth_PSAT_Math_Grade_10_School_Lbl,Growth_SAT_Reading_Grade_11_School_Pct,Growth_SAT_Reading_Grade_11_School_Lbl,Growth_SAT_Math_Grade_11_School_Pct,Growth_SAT_Math_Grade_11_School_Lbl
0,610547,SOUTH SHORE INTL HS,South Shore Intl College Prep High School,Selective enrollment,HS,1955 E 75TH ST,Chicago,Illinois,60649.0,7.735358e+09,NaN,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.southshoreinternational.org,2015.0,NaN,NaN,NaN,2012.0,NaN,BELOW AVERAGE,Student Growth measures the change in standard...,NaN,NaN,NaN,NaN,BELOW AVERAGE,Student Attainment measures how well the schoo...,NaN,NaN,NaN,NaN,ORGANIZED,Results are based on student and teacher respo...,81.1,81.2,90.5,80.7,NOT CERTIFIED,This school has not achieved Healthy School Ce...,INCOMPLETE DATA,...,68.1,72.3,63.0,71.9,"2,017","2,018",http://iirc.niu.edu/School.aspx?schoolid=15016...,2.9,45.4,NaN,This school does not have enough data for a re...,NOT RATED,This school has not submitted an action plan t...,"2,018",41.758792,-87.576353,818,891,929,969.0,9.0,9th,92.0,92nd,5.0,5th,35.7,35.7,45.9,45.9,18.5,18.5,34.1,34.1,29.0,29th,33.0,33rd,37.0,37th
1,400097,NOBLE - BULLS HS,Noble - Chicago Bulls College Prep,Charter,HS,2040 W ADAMS ST,Chicago,Illinois,60612.0,7.735348e+09,3.128500e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.chicagobullscollegeprep.org,2015.0,NaN,NaN,NaN,NaN,NaN,AVERAGE,Student Growth measures the change in standard...,NaN,NaN,NaN,NaN,ABOVE AVERAGE,Student Attainment measures how well the schoo...,NaN,NaN,NaN,NaN,WELL ORGANIZED,Results are based on student and teacher respo...,59.3,81.2,72.4,80.7,NOT CERTIFIED,This school has not achieved Healthy School Ce...,INCOMPLETE DATA,...,8

In [ ]:
full_df.shape[0]*full_df.shape[1]

444768

In [ ]:
cps_1819_df.isna().sum()

School_ID                                   0
Short_Name                                  0
Long_Name                                   0
School_Type                                 0
Primary_Category                            0
                                         ... 
Growth_PSAT_Math_Grade_10_School_Lbl      522
Growth_SAT_Reading_Grade_11_School_Pct    519
Growth_SAT_Reading_Grade_11_School_Lbl    519
Growth_SAT_Math_Grade_11_School_Pct       519
Growth_SAT_Math_Grade_11_School_Lbl       519
Length: 182, dtype: int64

In [ ]:
#Let's look at the columns first
show_all(cps_1819_df.columns)

School_ID, Short_Name, Long_Name, 
School_Type, Primary_Category, Address, 
City, State, Zip, 
Phone, Fax, CPS_School_Profile, 
Website, Progress_Report_Year, Blue_Ribbon_Award_Year, 
Excelerate_Award_Gold_Year, Spot_Light_Award_Year, Improvement_Award_Year, 
Excellence_Award_Year, Student_Growth_Rating, Student_Growth_Description, 
Growth_Reading_Grades_Tested_Pct_ES, Growth_Reading_Grades_Tested_Label_ES, Growth_Math_Grades_Tested_Pct_ES, 
Growth_Math_Grades_Tested_Label_ES, Student_Attainment_Rating, Student_Attainment_Description, 
Attainment_Reading_Pct_ES, Attainment_Reading_Lbl_ES, Attainment_Math_Pct_ES, 
Attainment_Math_Lbl_ES, Culture_Climate_Rating, Culture_Climate_Description, 
School_Survey_Student_Response_Rate_Pct, School_Survey_Student_Response_Rate_Avg_Pct, School_Survey_Teacher_Response_Rate_Pct, 
School_Survey_Teacher_Response_Rate_Avg_Pct, School_Survey_Parent_Response_Rate_Pct, School_Survey_Parent_Response_Rate_Avg_Pct, 
Healthy_School_Certification, Healthy_Schoo

In [ ]:
[col for col in cps_1819_df.columns if "graduation" in col.lower()]

['Graduation_4_Year_School_Pct_Year_2',
 'Graduation_4_Year_CPS_Pct_Year_2',
 'Graduation_4_Year_School_Pct_Year_1',
 'Graduation_4_Year_CPS_Pct_Year_1',
 'Graduation_5_Year_School_Pct_Year_2',
 'Graduation_5_Year_CPS_Pct_Year_2',
 'Graduation_5_Year_School_Pct_Year_1',
 'Graduation_5_Year_CPS_Pct_Year_1',
 'Progress_Toward_Graduation_Year_1',
 'Progress_Toward_Graduation_Year_2']

In [ ]:
corr = cps_1819_df.corr()
n = 20
print(corr['Graduation_4_Year_School_Pct_Year_2'].sort_values()[-n:])
print("")
print(corr['Graduation_4_Year_School_Pct_Year_2'].sort_values()[0:n])

NWEA_Math_Attainment_Grade_5_Pct        0.724554
Student_Attendance_Year_2_Pct           0.729513
NWEA_Math_Attainment_Grade_2_Pct        0.774278
NWEA_Reading_Growth_Grade_4_Pct         0.788195
NWEA_Reading_Attainment_Grade_4_Pct     0.801433
College_Enrollment_School_Pct_Year_1    0.822193
College_Enrollment_School_Pct_Year_2    0.859361
Graduation_5_Year_School_Pct_Year_1     0.928383
NWEA_Reading_Growth_Grade_3_Pct         0.939690
Graduation_5_Year_School_Pct_Year_2     0.950945
Graduation_4_Year_School_Pct_Year_1     0.955821
NWEA_Math_Attainment_Grade_4_Pct        0.967238
Graduation_4_Year_School_Pct_Year_2     1.000000
Excelerate_Award_Gold_Year                   NaN
Excellence_Award_Year                        NaN
Misconducts_To_Suspensions_Avg_Pct           NaN
Teacher_Attendance_Avg_Pct                   NaN
Graduation_5_Year_CPS_Pct_Year_1             NaN
College_Enrollment_CPS_Pct_Year_2            NaN
SAT_Grade_11_Score_CPS_Avg                   NaN
Name: Graduation_4_Y

In [ ]:
[col for col in cps_1819_df.head(1)]

['School_ID',
 'Short_Name',
 'Long_Name',
 'School_Type',
 'Primary_Category',
 'Address',
 'City',
 'State',
 'Zip',
 'Phone',
 'Fax',
 'CPS_School_Profile',
 'Website',
 'Progress_Report_Year',
 'Blue_Ribbon_Award_Year',
 'Excelerate_Award_Gold_Year',
 'Spot_Light_Award_Year',
 'Improvement_Award_Year',
 'Excellence_Award_Year',
 'Student_Growth_Rating',
 'Student_Growth_Description',
 'Growth_Reading_Grades_Tested_Pct_ES',
 'Growth_Reading_Grades_Tested_Label_ES',
 'Growth_Math_Grades_Tested_Pct_ES',
 'Growth_Math_Grades_Tested_Label_ES',
 'Student_Attainment_Rating',
 'Student_Attainment_Description',
 'Attainment_Reading_Pct_ES',
 'Attainment_Reading_Lbl_ES',
 'Attainment_Math_Pct_ES',
 'Attainment_Math_Lbl_ES',
 'Culture_Climate_Rating',
 'Culture_Climate_Description',
 'School_Survey_Student_Response_Rate_Pct',
 'School_Survey_Student_Response_Rate_Avg_Pct',
 'School_Survey_Teacher_Response_Rate_Pct',
 'School_Survey_Teacher_Response_Rate_Avg_Pct',
 'School_Survey_Parent_Respon

In [ ]:
target_df = cps_1819_df[['School_ID',
                        'Short_Name',
                        'Long_Name',
                        'School_Type',
                        'Suspensions_Per_100_Students_Year_2_Pct',
                        'Average_Length_Suspension_Year_2_Pct', 
                        'Behavior_Discipline_Year_2', 
                        'Chronic_Truancy_Pct',
                        'One_Year_Dropout_Rate_Year_2_Pct', 
                        'School_Survey_Safety',
                        'Student_Attainment_Rating', 
                        'Graduation_4_Year_School_Pct_Year_2',
                        'Teacher_Attendance_Year_2_Pct',]].dropna()

In [ ]:
target_df

,School_ID,Short_Name,Long_Name,School_Type,Suspensions_Per_100_Students_Year_2_Pct,Average_Length_Suspension_Year_2_Pct,Behavior_Discipline_Year_2,Chronic_Truancy_Pct,One_Year_Dropout_Rate_Year_2_Pct,School_Survey_Safety,Student_Attainment_Rating,Graduation_4_Year_School_Pct_Year_2,Teacher_Attendance_Year_2_Pct
0,609710,HARLAN HS,John M Harlan Community Academy High School,Neighborhood,38.8,2.1 days,"2,017",89.4,9.9,WEAK,FAR BELOW EXPECTATIONS,56.1,93.3
2,610334,RABY HS,Al Raby High School,Small,34.0,2.0 days,"2,017",85.7,2.2,WEAK,FAR BELOW EXPECTATIONS,68.5,95.0
9,609712,HIRSCH HS,Emil G Hirsch Metropolitan High School,Neighborhood,40.6,2.5 days,"2,017",100.8,14.1,NEUTRAL,FAR BELOW EXPECTATIONS,55.0,94.1
10,610543,SOLORIO HS,Eric Solorio Academy High School,Neighborhood,3.9,2.0 days,"2,017",34.1,3.4,WEAK,MET EXPECTATIONS,84.4,95.9
17,610389,ORR HS,Orr Academy High School,Neighborhood,20.5,2.2 days,"2,017",112.1,18.2,WEAK,FAR BELOW EXPECTATIONS,30.3,94.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,609738,LINCOLN PARK HS,Lincoln Park High School,Neighborhood,3.8,2.3 days,"2,017",28.5,3.3,WEAK,ABOVE EXPECTATIONS,87.6,94.8
600,609692,SIMEON HS,Neal F Simeon Career Academy High School,Career academy,23.3,2.6 days,"2,017",58.4,0.9,NEUTRAL,FAR BELOW EXPECTATIONS,87.8,94.1
602,609755,YOUNG HS,Whitney M Young Magnet High School,Selective enrollment,0.6,1.6 days,"2,017",1.8,0.7,NEUTRAL,FAR ABOVE EXPECTATIONS,97.0,95.0
618,610383,SOCIAL JUSTICE HS,Greater Lawndale High School For Social Justice,Small,16.5,2.2 days,"2,017",50.5,1.9,VERY WEAK,FAR BELOW EXPECTATIONS,86.0,94.3


In [ ]:
target_df['Student_Attainment_Rating'].value_counts()

FAR BELOW EXPECTATIONS    37
BELOW EXPECTATIONS        19
MET EXPECTATIONS          13
FAR ABOVE EXPECTATIONS     8
ABOVE EXPECTATIONS         5
Name: Student_Attainment_Rating, dtype: int64

In [ ]:
expec = []

for elem in target_df['Student_Attainment_Rating']:
    if elem == 'FAR BELOW EXPECTATIONS':
        expec.append(0)
    elif elem == 'BELOW EXPECTATIONS':
        expec.append(1)
    elif elem == 'MET EXPECTATIONS':
        expec.append(2)
    elif elem == 'ABOVE EXPECTATIONS':
        expec.append(3)
    elif elem == 'FAR ABOVE EXPECTATIONS':
        expec.append(4)

In [ ]:
target_df['Student_Attainment_Numeric'] = expec

In [ ]:
cps_1819_df.columns[0:13]

Index(['School_ID', 'Short_Name', 'Long_Name', 'School_Type',
       'Primary_Category', 'Address', 'City', 'State', 'Zip', 'Phone', 'Fax',
       'CPS_School_Profile', 'Website'],
      dtype='object')

In [ ]:
np.mean(target_df['Graduation_4_Year_School_Pct_Year_2'])

76.07439024390247

In [ ]:
target_df['Meeting_Graduation_Rates'] = [1 if elem >= 85 else 0 for elem in target_df.Graduation_4_Year_School_Pct_Year_2]

In [ ]:
target_df

,School_ID,Short_Name,Long_Name,School_Type,Suspensions_Per_100_Students_Year_2_Pct,Average_Length_Suspension_Year_2_Pct,Behavior_Discipline_Year_2,Chronic_Truancy_Pct,One_Year_Dropout_Rate_Year_2_Pct,School_Survey_Safety,Student_Attainment_Rating,Graduation_4_Year_School_Pct_Year_2,Teacher_Attendance_Year_2_Pct,Student_Attainment_Numeric,Meeting_Graduation_Rates
0,609710,HARLAN HS,John M Harlan Community Academy High School,Neighborhood,38.8,2.1 days,"2,017",89.4,9.9,WEAK,FAR BELOW EXPECTATIONS,56.1,93.3,0,0
2,610334,RABY HS,Al Raby High School,Small,34.0,2.0 days,"2,017",85.7,2.2,WEAK,FAR BELOW EXPECTATIONS,68.5,95.0,0,0
9,609712,HIRSCH HS,Emil G Hirsch Metropolitan High School,Neighborhood,40.6,2.5 days,"2,017",100.8,14.1,NEUTRAL,FAR BELOW EXPECTATIONS,55.0,94.1,0,0
10,610543,SOLORIO HS,Eric Solorio Academy High School,Neighborhood,3.9,2.0 days,"2,017",34.1,3.4,WEAK,MET EXPECTATIONS,84.4,95.9,2,0
17,610389,ORR HS,Orr Academy High School,Neighborhood,20.5,2.2 days,"2,017",112.1,18.2,WEAK,FAR BELOW EXPECTATIONS,30.3,94.3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,609738,LINCOLN PARK HS,Lincoln Park High School,Neighborhood,3.8,2.3 days,"2,017",28.5,3.3,WEAK,ABOVE EXPECTATIONS,87.6,94.8,3,1
600,609692,SIMEON HS,Neal F Simeon Career Academy High School,Career academy,23.3,2.6 days,"2,017",58.4,0.9,NEUTRAL,FAR BELOW EXPECTATIONS,87.8,94.1,0,1
602,609755,YOUNG HS,Whitney M Young Magnet High School,Selective enrollment,0.6,1.6 days,"2,017",1.8,0.7,NEUTRAL,FAR ABOVE EXPECTATIONS,97.0,95.0,4,1
618,610383,SOCIAL JUSTICE HS,Greater Lawndale High School For Social Justice,Small,16.5,2.2 days,"2,017",50.5,1.9,VERY WEAK,FAR BELOW EXPECTATIONS,86.0,94.3,0,1


In [ ]:
target_df.corr()

,School_ID,Suspensions_Per_100_Students_Year_2_Pct,Chronic_Truancy_Pct,One_Year_Dropout_Rate_Year_2_Pct,Graduation_4_Year_School_Pct_Year_2,Teacher_Attendance_Year_2_Pct,Student_Attainment_Numeric,Meeting_Graduation_Rates
School_ID,1.000000,-0.052087,0.027379,-0.037827,0.091651,0.284864,-0.144689,0.147596
Suspensions_Per_100_Students_Year_2_Pct,-0.052087,1.000000,0.659033,0.455625,-0.505841,-0.352396,-0.574467,-0.333530
Chronic_Truancy_Pct,0.027379,0.659033,1.000000,0.615890,-0.733106,-0.406574,-0.710178,-0.502690
One_Year_Dropout_Rate_Year_2_Pct,-0.037827,0.455625,0.615890,1.000000,-0.715714,-0.395932,-0.480904,-0.515927
Graduation_4_Year_School_Pct_Year_2,0.091651,-0.505841,-0.733106,-0.715714,1.000000,0.442656,0.692244,0.739570
Teacher_Attendance_Year_2_Pct,0.284864,-0.352396,-0.406574,-0.395932,0.442656,1.000000,0.393140,0.268015
Student_Attainment_Numeric,-0.144689,-0.574467,-0.710178,-0.480904,0.692244,0.393140,1.000000,0.596620
Meeting_Graduation_Rates,0.147596,-0.333530,-0.502690,-0.515927,0.739570,0.268015,0.596620,1.000000


In [ ]:
f_target_df = target_df[['Suspensions_Per_100_Students_Year_2_Pct', 'Average_Length_Suspension_Year_2_Pct', 'Chronic_Truancy_Pct', 'One_Year_Dropout_Rate_Year_2_Pct', 'Teacher_Attendance_Year_2_Pct',  'Student_Attainment_Numeric', 'Meeting_Graduation_Rates']]
target = target_df['Meeting_Graduation_Rates']

In [ ]:
X = f_target_df.drop('Meeting_Graduation_Rates', axis = 1)
y = f_target_df['Meeting_Graduation_Rates']

In [ ]:
len(f_target_df)

82

In [ ]:
#Target DF
f_target_df['Average_Length_Suspension_Year_2_Pct'] = [float(elem.split(" ")[0]) for elem in f_target_df['Average_Length_Suspension_Year_2_Pct']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
try:
    f_target_df['Average_Length_Suspension_Year_2_Pct'] = [float(elem[0:3]) for elem in f_target_df['Average_Length_Suspension_Year_2_Pct']]
except:
    print("(Already Casted as int)")

(Already Casted as int)


In [ ]:
#SK Learn
import sklearn
from sklearn.preprocessing import StandardScaler

X_features = X
X = StandardScaler().fit_transform(X)

ValueError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

model1 = DecisionTreeClassifier(random_state=1)
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)
print(classification_report(y_test, y_pred1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier(random_state=1)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
print(classification_report(y_test, y_pred2))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model3 = AdaBoostClassifier(random_state=1)
model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)
print(classification_report(y_test, y_pred3))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model4 = GradientBoostingClassifier(random_state=1)
model4.fit(X_train, y_train)
y_pred4 = model4.predict(X_test)
print(classification_report(y_test, y_pred4))

In [ ]:
y_pred4

In [ ]:
import xgboost as xgb
model5 = xgb.XGBClassifier(random_state=1)
model5.fit(X_train, y_train)
y_pred5 = model5.predict(X_test)
print(classification_report(y_test, y_pred5))

In [ ]:
# Filtering df for only good quality
df_temp = f_target_df[f_target_df['Meeting_Graduation_Rates']==1]
df_temp.describe()

In [ ]:
# Filtering df for only bad quality
df_temp2 = f_target_df[f_target_df['Meeting_Graduation_Rates']==0]
df_temp2.describe()

In [ ]:
import matplotlib
corr = f_target_df.corr()
matplotlib.pyplot.subplots(figsize=(15,10))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
#Clustering
f_target_df.head()

In [ ]:
f_target_df

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

from sklearn.datasets import make_blobs


from sklearn.metrics import accuracy_score

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score
from sklearn.metrics import v_measure_score, completeness_score, homogeneity_score
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph

In [ ]:
f_target_df.columns

In [ ]:
km_cluster = KMeans(3, random_state=9) # We know there are 3 class labels, lets see how well we can seperate them with clustering

In [ ]:
km_clusters = km_cluster.fit_predict(f_target_df.drop('One_Year_Dropout_Rate_Year_2_Pct',axis=1))

In [ ]:
km_clusters

In [ ]:
time.time() - start_time()

In [ ]:
print("Homogeneity", homogeneity_score(f_target_df['One_Year_Dropout_Rate_Year_2_Pct'], km_clusters) ) 
print("Completeness", completeness_score(f_target_df['One_Year_Dropout_Rate_Year_2_Pct'], km_clusters) ) 
print("V-Score", v_measure_score(f_target_df['One_Year_Dropout_Rate_Year_2_Pct'], km_clusters) ) 